# Amazon Spark Analysis

## Dependancies

### Import Spark Dependancies

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu bionic-sec

### Import SQL Dependancies

In [2]:
# First, enter the following code to download a Postgres driver that will allow Spark to interact with Postgres:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

# Then, start a Spark session with an additional option that adds the driver to Spark:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

--2022-07-13 20:58:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.4’

postgresql-42.2.16. 100%[===================>] 979.38K  5.49MB/s    in 0.2s    

2022-07-13 20:58:01 (5.49 MB/s) - ‘postgresql-42.2.16.jar.4’ saved [1002883/1002883]



## Tables

### Load Data from .tsv

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://beanies-bucket.s3.us-west-1.amazonaws.com/amazon_reviews_us_Video_Games_v1_00.tsv"
spark.sparkContext.addFile(url)
video_games_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv"), sep="\t", 
                                        header=True, inferSchema=True)
video_games_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Customers Table

In [4]:
customers_table = video_games_reviews_df.groupby(['customer_id']).count().withColumnRenamed('count', 'customer_count')

customers_table.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



### Product Table

In [5]:
product_table = video_games_reviews_df.select(['product_id','product_title']).drop_duplicates()

product_table.show(truncate = False)

+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                                           |
+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|B00CJ7IUI6|The Elder Scrolls V: Skyrim Legendary Edition                                                                                                                                                                           |
|B00DHF39KS|Wolfenstein: The New Order                                          

### Review ID Table

In [6]:
from pyspark.sql.functions import *
review_id_table = video_games_reviews_df.select(['review_id','customer_id', 'product_id','product_parent', 
                                                 to_date('review_date', 'yyyy-MM-dd').alias('review_date')])

review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-08

### Vine Table

In [7]:
vine_table = video_games_reviews_df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])

vine_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

### Combined Tables

In [8]:
# Filter rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
greaterThan20 = customers_table.filter(customers_table.customer_count>=20)
greaterThan20.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16229582|            24|
|   17481726|            95|
|   11272310|            20|
|   14175902|            22|
|    6367756|            65|
|    7455053|            50|
|   41605744|            21|
|   31564043|            24|
|   35892611|            29|
|   17171509|            20|
|   46595148|            28|
|   53067883|            53|
|   51402733|            20|
|   47823593|            44|
|   51680053|            31|
|   49336508|            21|
|   14365065|            27|
|   35474593|            20|
|   23384571|            23|
|   16298267|            22|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of
# helpful_votes divided by total_votes is equal to or greater than 50%.
goodVotes = vine_table.filter((vine_table.helpful_votes)/(vine_table.total_votes)>=.5)
fullReviews = goodVotes.join(review_id_table, vine_table.review_id == review_id_table.review_id)
analysisTable = greaterThan20.join(fullReviews, greaterThan20.customer_id == fullReviews.customer_id)
analysisTable.show()

+-----------+--------------+--------------+-----------+-------------+-----------+----+-----------------+--------------+-----------+----------+--------------+-----------+
|customer_id|customer_count|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_id|customer_id|product_id|product_parent|review_date|
+-----------+--------------+--------------+-----------+-------------+-----------+----+-----------------+--------------+-----------+----------+--------------+-----------+
|    6367756|            65|R17T28Y9WKGKYN|          5|            1|          1|   N|                Y|R17T28Y9WKGKYN|    6367756|B00009VRFX|     523524339| 2014-11-20|
|    6367756|            65| R6334LOK39I13|          4|            1|          1|   N|                Y| R6334LOK39I13|    6367756|B001NQB6BW|     744092472| 2014-11-20|
|    6367756|            65|R13AH2G8X0P20T|          4|            1|          1|   N|                Y|R13AH2G8X0P20T|    6367756|B00007L4WE|     195

In [10]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of 
# review (paid vs unpaid).
import pandas as pd

# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was
# written as part of the Vine program (paid), vine == 'Y'.
vine = analysisTable.filter(analysisTable.vine == 'Y')

total_count = vine.count()
rating_count = vine.filter(vine.star_rating == 5).count()
percentage =  f'{(rating_count/total_count)*100}%'
vineDescribed = pd.DataFrame(data={'Total Count': total_count, '5 Star Rating Count': rating_count, 
                              '5 Star Percentage': percentage}, index=[0])
vineDescribed.head()

,Total Count,5 Star Rating Count,5 Star Percentage
0,463,163,35.20518358531318%


In [11]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
notVine = analysisTable.filter(analysisTable.vine == 'N')

total_count = notVine.count()
rating_count = notVine.filter(notVine.star_rating == 5).count()
percentage =  f'{(rating_count/total_count)*100}%'
notVineDescribed = pd.DataFrame(data={'Total Count': total_count, '5 Star Rating Count': rating_count, 
                              '5 Star Percentage': percentage}, index=[0])
notVineDescribed.head()

,Total Count,5 Star Rating Count,5 Star Percentage
0,50749,25029,49.3191984078504%


## Upload to Postgre Database

In [12]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.ccifjjodkw3d.us-west-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", "password": password, "driver":"org.postgresql.Driver"}

Enter database password··········


In [41]:
# Customers Table
customers_table.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [42]:
# Product Table
product_table.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [43]:
# Review ID Table
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [44]:
# Vine Table
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)